In [2]:
from bs4 import BeautifulSoup as bs, Comment
import requests
import pandas as pd
pd.set_option('display.max_columns', None)



In [3]:
url = "https://fbref.com/en/comps/12/stats/La-Liga-Stats"
headers = {'User-Agent': 'Mozilla/5.0'} 
response = requests.get(url, headers=headers)

In [4]:
soup = bs(response.text)
type(soup)
soup.find('title').text
print(response.status_code)



200


In [5]:
#extracting the headers
laliga_table = soup.find('table', id="stats_squads_standard_for")
laliga_thead = laliga_table.find('thead')

laliga_header = laliga_thead.find_all('th')
#print(header)
columns = [ele.text.strip() for ele in laliga_header]
#print(columns)


#creating subheaders and headers
n = 6 
over_header = columns[:n]
under_header = columns[n:]
over_header_replaced = over_header[0]="Empty"

#structure of the subheader and header
structure = {
    'Empty': 4,
    'Playing Time': 4,
    'Performance': 8,
    'Expected': 4,
    'Progression': 2,
    'Per 90 Minutes': 10
}

over_headers =[]
for key, count in structure.items():
    over_headers.extend([key]*count)

#creating the subheader and header
multi_index = pd.MultiIndex.from_arrays([over_headers, under_header])






In [6]:
# extracting the rows
laliga_tbody = laliga_table.find('tbody')


rows = laliga_tbody.find_all('tr')
d=[]
for row in rows:
    columns = row.find_all(['td', 'th'])
    row = [ele.text.strip() for ele in columns]
    d.append((row))
    

laliga_squad_df = pd.DataFrame(d, columns=multi_index)

    

In [7]:
laliga_table_player = soup.find('table', id="stats_standard")
print(laliga_table_player)
laliga_table = soup.find('table', id="stats_squads_standard_for")
laliga_thead = laliga_table.find('thead')




None


In [8]:
# Look through all comments to find the table
for comment in soup.find_all(string=lambda text: isinstance(text, Comment)):
    if 'stats_standard' in comment:
        comment_soup = bs(comment, 'html.parser')
        laliga_table_player = comment_soup.find('table', id='stats_standard')

laliga_thead_player = laliga_table_player.find('thead')
laliga_header_player = laliga_thead_player.find_all('th')
columns2 = [ele.text.strip() for ele in laliga_header_player]
#print(columns)

n = 7

over_header2 = columns2[:n]
under_header2 = columns2[n:]
over_header_replaced2 = over_header2[0]="Empty"
over_header_replaced_end = over_header2[6]="Matches"

#structure of the subheader and header
structure2 = {
    'Empty': 7,
    'Playing Time': 4,
    'Performance': 8,
    'Expected': 4,
    'Progression': 3,
    'Per 90 Minutes': 10,
    'Matches': 1
}

over_headers2 =[]
for key, count in structure2.items():
    over_headers2.extend([key]*count)

#creating the subheader and header
multi_index2 = pd.MultiIndex.from_arrays([over_headers2, under_header2])

print(multi_index2)

MultiIndex([(         'Empty',       'Rk'),
            (         'Empty',   'Player'),
            (         'Empty',   'Nation'),
            (         'Empty',      'Pos'),
            (         'Empty',    'Squad'),
            (         'Empty',      'Age'),
            (         'Empty',     'Born'),
            (  'Playing Time',       'MP'),
            (  'Playing Time',   'Starts'),
            (  'Playing Time',      'Min'),
            (  'Playing Time',      '90s'),
            (   'Performance',      'Gls'),
            (   'Performance',      'Ast'),
            (   'Performance',      'G+A'),
            (   'Performance',     'G-PK'),
            (   'Performance',       'PK'),
            (   'Performance',    'PKatt'),
            (   'Performance',     'CrdY'),
            (   'Performance',     'CrdR'),
            (      'Expected',       'xG'),
            (      'Expected',     'npxG'),
            (      'Expected',      'xAG'),
            (      'Expected', '

In [9]:
laliga_tbody_player = laliga_table_player.find('tbody')

rows2 = laliga_tbody_player.find_all('tr')
f=[]
for row in rows2:
    columns = row.find_all(['td','th'])
    row = [ele.text.strip() for ele in columns]
    f.append((row))

laliga_players_df = pd.DataFrame(f, columns=multi_index2)

In [10]:
laliga_players_df['League'] = "La Liga"
laliga_players_df['Season'] = "2024/25"
laliga_players_df.to_csv('../csv_files/laliga_players_2024_25.csv', index=False)